# Load the Packages

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RNN, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from keras import backend as K

# Define Confusion Matrix

In [ ]:
tf.compat.v1.get_default_graph()


def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# Load and Prepare the Data

In [ ]:

df = pd.read_excel(r"\Payload_4year_trend.xlsx",sheet_name='Train')
test_set = pd.read_excel(r"\Payload_4year_trend.xlsx",sheet_name='Test')

#df=Normalized(df)
#test_set=Normalized (test_set)

real_test = test_set.iloc[:,1:2].values
input_data1= df.iloc[:,1:2].values
sc = MinMaxScaler()
input_data = sc.fit_transform(input_data1)

X_train = input_data[0:1493] # 7 shows we predict 7 days 
Y_train = input_data[7:1500] # 7 shows we predict 7 days 

X_test = real_test[0:169]
Y_test= real_test[7:176]

X_train = np.reshape(X_train,(1493,1,1))
X_test = np.reshape(X_test,(169,1,1))


# To evaluate each "Activation function", "Loss Function", and "Optimizer" have the best result, all these parameters have been defined and fitted to the RNN to compare the results

In [ ]:

activation_function=['sigmoid','relu','tanh','softmax']
Optimizer_=['sgd','RMSprop','Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
Loos_function=['squared_hinge','hinge','categorical_hinge','logcosh','mean_absolute_error','mean_squared_error','mean_absolute_percentage_error','mean_squared_logarithmic_error']

RMSE=['Activation function','optimizer','RMSE']
evalution=pd.DataFrame([['Activation function','optimizer','ERROR','loss', 'accuracy', 'f1_score', 'precision', 'recall']])


In [ ]:
for j in Optimizer_:
    for i in activation_function:
        for k in Loos_function:
            RNN_Model = Sequential()
            RNN_Model.add(GRU(units=40,activation=i,input_shape=(None,1)))
            RNN_Model.add(Dropout(0.1))
            RNN_Model.add(Dense(units=1))
            RNN_Model.compile(optimizer = j, loss = k, metrics=['accuracy', f1_m,precision_m, recall_m])
            History=RNN_Model.fit(X_train, Y_train, batch_size = 10, epochs = 300) 
            epoch=1    
            inputs = real_test
            inputs = sc.transform(inputs)
            inputs = np.reshape(inputs, (63, 1, 1))
            predicted = RNN_Model.predict(inputs)
            predicted = sc.inverse_transform(predicted)

            RNN_Model.predict_generator
            plt.plot(real_test, color = 'red', label = 'Real ')
            plt.plot(predicted, color = 'blue', label = 'Predicted ')
            title_='Activation function= '+i+'   optimizer='+j+'Loos_Function= '+k
            plt.title(title_)
            plt.xlabel('Time')
            plt.ylabel('GByte')
            plt.legend()
            fig = plt.gcf()
            fig.set_size_inches(10, 6)
            plt.show()
            rmse = math.sqrt(mean_squared_error(real_test, predicted))
            RMSE.append([i,j,rmse])
            print ("Error: "+str(rmse))
            evalu=pd.DataFrame([[i, j, k, rmse, History.history['loss'][epoch], History.history['accuracy'][epoch], History.history['f1_m'][epoch], History.history['precision_m'][epoch], History.history['recall_m'][epoch]]])
            evalution=evalution.append(evalu)

        print(evalu)



# Save to an Excel file

In [43]:
evalu=pd.DataFrame(evalu)
writer = pd.ExcelWriter('evalu.xlsx')   
evalu.to_excel(writer,'result')

writer.save()